In [2]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import datetime
import os

In [5]:
fechas = [i for i in os.listdir('datos_cirugias/') if len(i) == 12]
fechas.sort()

In [6]:
prestador = 68761

In [7]:
temporal_prestador = pd.DataFrame(columns= ["cirugías"], data= ["porcentaje de cirugías en centros propios", "porcentaje de cirugías en centros no propios"])
temporal_total = pd.DataFrame(columns= ["cirugías"], data= ["porcentaje de cirugías en centros propios", "porcentaje de cirugías en centros no propios"])

In [10]:
for i in fechas:
    final = pd.read_parquet(f'datos_cirugias/{i}') 
    if len(final[final["Prestador"] == prestador]) == 0:
        temporal_total[i[:7]] = [np.nan]*2
        temporal_prestador[i[:7]] =  [np.nan]*2
    else: 
        propios = final["Propias"].sum()
        no_propios = final["No propias"].sum()
        total = propios + no_propios
        ar = [propios / total , no_propios /total]
        temporal_total[i[:7]] = ar
        final_prestador = final[final["Prestador"] == prestador]
        final_prestador['porcentaje de cirugías en centros propios'] = final_prestador["Propias"] / final_prestador["Total general"]
        final_prestador['porcentaje de cirugías en centros no propios' ] = final_prestador["No propias"] / final_prestador["Total general"]
        temporal_prestador[i[:7]] = final_prestador[["porcentaje de cirugías en centros no propios", "porcentaje de cirugías en centros propios"]].values[0]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
temporal_prestador

,cirugías,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
0,porcentaje de cirugías en centros propios,0.0,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN
1,porcentaje de cirugías en centros no propios,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN


In [15]:
temporal_total

,cirugías,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02
0,porcentaje de cirugías en centros propios,0.778612,NaN,NaN,0.773503,NaN,0.780718,NaN,0.780963,0.782203,0.79451,0.763047,0.804136,NaN
1,porcentaje de cirugías en centros no propios,0.221388,NaN,NaN,0.226497,NaN,0.219282,NaN,0.219037,0.217797,0.20549,0.236953,0.195864,NaN


In [16]:
y_prestador

array([0.0, nan, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, 0.0, 0.0, 0.0, nan],
      dtype=object)

In [17]:
x = temporal_prestador.columns[1:]
y_prestador = temporal_prestador[:1].values[0][1:]
y_total =  temporal_total[:1].values[0][1:]
fig_cirugias = go.Figure()
fig_cirugias.add_trace(go.Bar(x = x ,
                         y = y_prestador,
                         name = "Prestador",
                         marker_color ="#45b6fe"))
fig_cirugias.add_trace(go.Scatter(x = x ,
                         y = y_total,
                         name = "Cartilla Swiss Medical Medicina Privada",
                         marker_color ="#3792cb"))
fig_cirugias.layout.legend = {"orientation": "h", "xanchor" :"center", "y":1.14, "x":0.5}
fig_cirugias.update_yaxes(range=[-0.1,1],tickformat = "%", title="Porcentaje de cirugías en centros propios")
fig_cirugias.show()